In [16]:
import os,sys
sys.path.append("..")
import numpy as np
import pandas as pd
import ibmseti
import collections
import scipy.io
import matplotlib.pyplot as plt
import commonutils as cu
import PIL
from PIL import Image

In [17]:
import keras
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [29]:
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from model_specs import vgg_fine_tune

In [33]:
# Might want to tune these parameters
train_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,
                                   rotation_range=0, width_shift_range=0.2,
                                   height_shift_range=0.1,zoom_range=0.1,
                                   horizontal_flip=True, vertical_flip = True, fill_mode='constant')
test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)

# # Create a sample dataset array for fitting whitening / normalization parameters
# #(linux command for putting all images in one place) 
# # find train -name '*.jpg' -type f -exec cp {} . \;
trainImgDir = '../data/imagesDataset_512x256_8/train_one_folder/'
files = [f for f in os.listdir(trainImgDir) if f.endswith('.jpg')]
size = img_to_array(load_img(os.path.join(trainImgDir,files[0]),grayscale=False)).shape

In [38]:
print size

(256, 512, 3)


In [35]:
model = vgg_fine_tune.build(input_shape= (size[0],size[1],size[2]),nb_classes=4)

model_specs.py:33: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, kernel_initializer="he_normal", activation="relu", input_shape=(65536,))`
  activation='relu',init="he_normal"))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 128, 128)      0         
__________

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
# model.fit(train_data, train_labels,
#           epochs=50,
#           batch_size=batch_size,
#           validation_data=(validation_data, validation_labels))

In [40]:
trainDataArray = np.zeros((len(files)//5,)+size) # Only 1/10 the data, because memory constraints
for i in range(len(files)//5):
    trainDataArray[i] = img_to_array(load_img(os.path.join(trainImgDir,files[i])))
    
train_datagen.fit(trainDataArray)
test_datagen.fit(trainDataArray)

datasetPath = "../data/imagesDataset_512x256_8/train/"
valDataPath = "../data/imagesDataset_512x256_8/validation/"
classList = ['0-noise','2-narrowband','3-narrowbanddrd','5-squiggle']
batch_size = 8
train_generator = train_datagen.flow_from_directory(directory=datasetPath,batch_size=batch_size,class_mode='categorical',
                            classes=classList,target_size=(256,512))
validation_generator = test_datagen.flow_from_directory(directory=valDataPath,batch_size=batch_size,
        class_mode='categorical',classes=classList,target_size=(256,512))

Found 7584 images belonging to 4 classes.
Found 1498 images belonging to 4 classes.


In [41]:
epochs = 20
nb_train_samples = 3000
nb_validation_samples = 800
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=1)


Epoch 1/20


InternalError: Dst tensor is not initialized.
	 [[Node: Const_21 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [65536,1024] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'Const_21', defined at:
  File "/home/cs341seti/anaconda3/envs/cs341seti/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/cs341seti/anaconda3/envs/cs341seti/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/cs341seti/anaconda3/envs/cs341seti/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/cs341seti/anaconda3/envs/cs341seti/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/cs341seti/anaconda3/envs/cs341seti/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/

In [ ]:
model.predict_generator(validation_generator,verbose=1,validation_steps=nb_validation_samples// batch_size)

In [ ]:
# dimensions of our images.

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)